#Basics

In [1]:
!pip install nvcc4jupyter

In [2]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmprduvcdeu".


In [7]:
%%cuda

#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

__host__ int main(){
    hello<<<3, 3>>>();
    cudaDeviceSynchronize();
}

Hello from block: 1, thread: 0
Hello from block: 1, thread: 1
Hello from block: 1, thread: 2
Hello from block: 2, thread: 0
Hello from block: 2, thread: 1
Hello from block: 2, thread: 2
Hello from block: 0, thread: 0
Hello from block: 0, thread: 1
Hello from block: 0, thread: 2



In [8]:
%%cuda
#include <stdio.h>

__global__ void hello() {
    printf("Hello from block: (%u, %u, %u), thread: (%u, %u, %u)\n",
           blockIdx.x, blockIdx.y, blockIdx.z,
           threadIdx.x, threadIdx.y, threadIdx.z);
}

int main() {
    // Define the dimensions of the grid and blocks
    dim3 gridDim(2, 2, 2);   // 2x2x2 grid of blocks
    dim3 blockDim(2, 2, 2);  // 2x2x2 grid of threads per block

    // Launch the kernel
    hello<<<gridDim, blockDim>>>();

    // Wait for GPU to finish before accessing on host
    cudaDeviceSynchronize();

    return 0;
}

Hello from block: (1, 0, 0), thread: (0, 0, 0)
Hello from block: (1, 0, 0), thread: (1, 0, 0)
Hello from block: (1, 0, 0), thread: (0, 1, 0)
Hello from block: (1, 0, 0), thread: (1, 1, 0)
Hello from block: (1, 0, 0), thread: (0, 0, 1)
Hello from block: (1, 0, 0), thread: (1, 0, 1)
Hello from block: (1, 0, 0), thread: (0, 1, 1)
Hello from block: (1, 0, 0), thread: (1, 1, 1)
Hello from block: (0, 1, 1), thread: (0, 0, 0)
Hello from block: (0, 1, 1), thread: (1, 0, 0)
Hello from block: (0, 1, 1), thread: (0, 1, 0)
Hello from block: (0, 1, 1), thread: (1, 1, 0)
Hello from block: (0, 1, 1), thread: (0, 0, 1)
Hello from block: (0, 1, 1), thread: (1, 0, 1)
Hello from block: (0, 1, 1), thread: (0, 1, 1)
Hello from block: (0, 1, 1), thread: (1, 1, 1)
Hello from block: (0, 0, 1), thread: (0, 0, 0)
Hello from block: (0, 0, 1), thread: (1, 0, 0)
Hello from block: (0, 0, 1), thread: (0, 1, 0)
Hello from block: (0, 0, 1), thread: (1, 1, 0)
Hello from block: (0, 0, 1), thread: (0, 0, 1)
Hello from bl

In [9]:
%%cuda
#include <iostream>
#include <ctime>
#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>

using namespace std;

//brute force approach to finding which point
void findClosestCPU(float3* points, int* indices, int count) {
    // Base case, if there's 1 point don't do anything
    if(count <=1) return;
    // Loop through every point
    for (int curPoint = 0; curPoint < count; curPoint++) {
        // set as close to the largest float possible
        float distToClosest = 3.4028238f ;
        // See how far it is from every other point
        for (int i = 0; i < count; i++) {
            // Don't check distance to itself
            if(i == curPoint) continue;
            float dist_sqr = (points[curPoint].x - points[i].x) *
                (points[curPoint].x - points[i].x) +
                (points[curPoint].y - points[i].y) *
                (points[curPoint].y - points[i].y) +
                (points[curPoint].z - points[i].z) *
                (points[curPoint].z - points[i].z);
            if(dist_sqr < distToClosest) {
                distToClosest = dist_sqr;
                indices[curPoint] = i;
            }
        }
    }
}

int main(){

    //defining parameters
    const int count = 10000;
    int* indexOfClosest = new int[count];
    float3* points = new float3[count];

    //defining random points
    for (int i = 0; i < count; i++){
        points[i].x = (float)(((rand()%10000))-5000);
        points[i].y = (float)(((rand()%10000))-5000);
        points[i].z = (float)(((rand()%10000))-5000);
    }

    long fastest = 1000000000;

    cout << "running brute force nearest neighbor on the CPU..."<<endl;
    for (int i = 0; i <= 10; i++){
        long start = clock();
        findClosestCPU(points, indexOfClosest, count);
        double duration = ( clock() - start ) / (double) CLOCKS_PER_SEC;
        cout << "test " << i << " took " << duration << " seconds" <<endl;
    }

    return 0;
}

running brute force nearest neighbor on the CPU...
test 0 took 1.2005 seconds
test 1 took 1.18997 seconds
test 2 took 1.18641 seconds
test 3 took 1.17282 seconds
test 4 took 1.91539 seconds
test 5 took 1.49837 seconds
test 6 took 1.18263 seconds
test 7 took 1.17336 seconds
test 8 took 1.21073 seconds
test 9 took 1.18849 seconds
test 10 took 1.17642 seconds



In [12]:
%%writefile findClosestGPU.cu
#include <iostream>
#include <ctime>
#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>

using namespace std;

// Brute force implementation, parallelized on the GPU
__global__ void findClosestGPU(float3* points, int* indices, int count) {
    if (count <= 1) return;
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < count) {
        float3 thisPoint = points[idx];
        float smallestSoFar = 3.40282e38f;

        for (int i = 0; i < count; i++) {
            if (i == idx) continue;

            float dist_sqr = (thisPoint.x - points[i].x) *
                             (thisPoint.x - points[i].x) +
                             (thisPoint.y - points[i].y) *
                             (thisPoint.y - points[i].y) +
                             (thisPoint.z - points[i].z) *
                             (thisPoint.z - points[i].z);
            if (dist_sqr < smallestSoFar) {
                smallestSoFar = dist_sqr;
                indices[idx] = i;
            }
        }
    }
}

int main() {
    // Defining parameters
    const int count = 10000;
    int* h_indexOfClosest = new int[count];
    float3* h_points = new float3[count];

    // Defining random points
    for (int i = 0; i < count; i++) {
        h_points[i].x = (float)(((rand() % 10000)) - 5000);
        h_points[i].y = (float)(((rand() % 10000)) - 5000);
        h_points[i].z = (float)(((rand() % 10000)) - 5000);
    }

    // Device pointers
    int* d_indexOfClosest;
    float3* d_points;

    // Allocating memory on the device
    cudaMalloc(&d_indexOfClosest, sizeof(int) * count);
    cudaMalloc(&d_points, sizeof(float3) * count);

    // Copying values from the host to the device
    cudaMemcpy(d_points, h_points, sizeof(float3) * count, cudaMemcpyHostToDevice);

    int threads_per_block = 64;
    cout << "Running brute force nearest neighbor on the GPU..." << endl;
    for (int i = 1; i <= 10; i++) {
        long start = clock();

        findClosestGPU<<<(count / threads_per_block) + 1, threads_per_block>>>(d_points, d_indexOfClosest, count);
        cudaDeviceSynchronize();

        // Copying results from the device to the host
        cudaMemcpy(h_indexOfClosest, d_indexOfClosest, sizeof(int) * count, cudaMemcpyDeviceToHost);

        double duration = (clock() - start) / (double)CLOCKS_PER_SEC;
        cout << "Test " << i << " took " << duration << " seconds" << endl;
    }

    // Freeing device memory
    cudaFree(d_indexOfClosest);
    cudaFree(d_points);

    // Freeing host memory
    delete[] h_indexOfClosest;
    delete[] h_points;

    return 0;
}

Writing findClosestGPU.cu


In [19]:
%%shell
nvprof ./findClosestGPU.out

==5284== NVPROF is profiling process 5284, command: ./findClosestGPU.out
Running brute force nearest neighbor on the GPU...
Test 1 took 0.00267 seconds
Test 2 took 0.002464 seconds
Test 3 took 0.002435 seconds
Test 4 took 0.002429 seconds
Test 5 took 0.002435 seconds
Test 6 took 0.002429 seconds
Test 7 took 0.002428 seconds
Test 8 took 0.00243 seconds
Test 9 took 0.002434 seconds
Test 10 took 0.002437 seconds
==5284== Profiling application: ./findClosestGPU.out
==5284== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.76%  23.830ms        10  2.3830ms  2.3798ms  2.3845ms  findClosestGPU(float3*, int*, int)
                    0.19%  45.760us        10  4.5760us  4.4480us  5.0240us  [CUDA memcpy DtoH]
                    0.05%  12.448us         1  12.448us  12.448us  12.448us  [CUDA memcpy HtoD]
      API calls:   79.38%  95.898ms         2  47.949ms  4.5110us  95.893ms  cudaMalloc
                   19.76%  23.871

# Building NN from Scratch

##preliminaries of C

In [1]:
%%writefile someClass.hh

// this is used so, if someClass gets imported multiple times across several
// documents, it only actually gets imported once.
#pragma once

class ClassWithFunctionality {
    // defining private things for internal use
private:
    // defining private data
    int someValue;
    int anotherValue;

    // defining private functions
    void privateFunction1();
    void privateFunction2();

    // defining things accessible outside the object
public:
    // defining public data
    int somePublicValue;
    int someOtherPublicValue;

    // defining public functions
    ClassWithFunctionality(int constructorInput);
    void doSomething1();
    void doSomething2();
};

Writing someClass.hh


In [2]:
%%writefile someClass.cu

#include "someClass.hh"

ClassWithFunctionality::ClassWithFunctionality(int constructorInput)
    : someValue(constructorInput), anotherValue(2), somePublicValue(3), someOtherPublicValue(4)
{}

void ClassWithFunctionality::doSomething1() {
    return;
}

void ClassWithFunctionality::doSomething2() {
    return;
}

void ClassWithFunctionality::privateFunction1() {
    return;
}

void ClassWithFunctionality::privateFunction2() {
    return;
}

Writing someClass.cu


In [3]:
%%writefile main.cu
#include <iostream>
#include "someClass.hh"

// testing SomeClass
int main(void) {
    ClassWithFunctionality example(3);
    std::cout << "it works!" << std::endl;
    return 0;
}

Writing main.cu


In [4]:
%%shell

nvcc someClass.cu main.cu -o main
./main

it works!


##main

###File: Shape

This is for matrices with a 2D shape

In [5]:
%%writefile shape.hh

#pragma once

struct Shape {
  size_t x, y;

  Shape(size_t x = 1, size_t y = 1);
};

Writing shape.hh


In [6]:
%%writefile shape.cu
#include "shape.hh"

Shape::Shape(size_t x, size_t y) :
 x(x), y(y)
{ }

Writing shape.cu


In [7]:
%%writefile main.cu
#include "shape.hh"
#include <iostream>
#include <stdio.h>

using namespace std;

int main( void ) {
  Shape shape = Shape(100, 200);
  cout << "x: " << shape.x << " y: " << shape.y << endl;
  return 0;
}

Overwriting main.cu


In [8]:
%%shell
nvcc shape.cu main.cu -o shape.out
./shape.out

x: 100 y: 200


###File: NNException

NNException is a lightweight wrapper built around cudaGetLastError that allows us to check, throughout our code, if there was an error on the GPU

In [9]:
%%writefile nn_exception.hh
#pragma once

#include <exception>
#include <iostream>

class NNException : std::exception {
  private:
    const char* exception_message;

  public:
    NNException(const char* exception_message) :
      exception_message(exception_message)
    { }

    virtual const char* what() const throw()
    {
      return exception_message;
    }

    static void throwIfDeviceErrorsOccurred(const char* exception_message){
      cudaError_t error = cudaGetLastError();
      if (error != cudaSuccess) {
        throw NNException(exception_message);
      }
    }
};


Writing nn_exception.hh


In [10]:
%%writefile main.cu

#include "nn_exception.hh"
#include <cuda_runtime.h>

int main() {
  float* d_data;
  cudaError_t error = cudaMalloc((void**)&d_data, 100*sizeof(float)); // allocate memory for the device = GPU

  try {
    NNException::throwIfDeviceErrorsOccurred("Failed to allocate GPU memry");
  }
  catch (const NNException& e) {
    std::cerr << e.what() << std::endl;
    return -1;
  }
  error = cudaFree(d_data);

  // Check for CUDA errors again
    try {
        NNException::throwIfDeviceErrorsOccurred("Failed to free GPU memory");
    } catch (const NNException& e) {
        std::cerr << "Caught NNException: " << e.what() << std::endl;
        return -1; // Return an error code
    }

    std::cout << "CUDA operations completed successfully" << std::endl;
    return 0; // Return success

}

Overwriting main.cu


In [11]:
%%shell
nvcc main.cu shape.cu -o nnexception.out
./nnexception.out

CUDA operations completed successfully


###File: Matrix

This class abstracts some of the communication between the device and host, allowing a matrix of values to easily be passed between memory locations. It allows for:

* memory to be allocated on the GPU for the matrix
* memory to be allocated on the CPU for the matrix
* memory to be allocated on both the CPU and GPU for the matrix
allocate memory, if it isn’t allocated already
* copy data from the CPU RAM to GPU VRAM
* copy data from the GPU VRAM to CPU RAM
* overrides to allow the matrix to be indexed like an array

In [12]:
%%writefile matrix.hh

#pragma once

#include "shape.hh"
#include <memory>

class Matrix {
  private:
    bool device_allocated;
    bool host_allocated;

    void allocateCudaMemory();
    void allocateHostMemory();

  public:
    Shape shape;
    std::shared_ptr<float> data_device;
    std::shared_ptr<float> data_host;

    Matrix(size_t x_dim = 1, size_t y_dim = 1);
    Matrix(Shape shape);

    void allocateMemory();
    void allocateMemoryIfNotAllocated(Shape shape);

    void copyHostToDevice();
    void copyDeviceToHost();

    float& operator[](const int index);
    const float& operator[](const int index) const;
};

Writing matrix.hh


In [13]:
%%writefile matrix.cu

#include "matrix.hh"
#include "nn_exception.hh"

using namespace std;

Matrix::Matrix(size_t x_dim, size_t y_dim) :
  shape(x_dim, y_dim),
  data_device(nullptr),
  data_host(nullptr),
  device_allocated(false),
  host_allocated(false)
{ }

Matrix::Matrix(Shape shape) :
  Matrix(shape.x, shape.y)
{ }

void Matrix::allocateCudaMemory() {
  if (!device_allocated) {
    float* device_memory = nullptr;
    cudaMalloc(&device_memory, shape.x * shape.y * sizeof(float));

    NNException::throwIfDeviceErrorsOccurred("Cannot allocate CUDA memory for tensor3D");
    data_device = std::shared_ptr<float>(device_memory, [&](float* ptr){ cudaFree(ptr);});
    device_allocated = true;
  }
}
void Matrix::allocateHostMemory() {
 if (!host_allocated) {
  data_host = std::shared_ptr<float>(new float[shape.x * shape.y],
             [&](float* ptr){ delete[] ptr; });
  host_allocated = true;
 }
}

void Matrix::allocateMemory() {
 allocateCudaMemory();
 allocateHostMemory();
}

void Matrix::allocateMemoryIfNotAllocated(Shape shape) {
 if (!device_allocated && !host_allocated) {
  this->shape = shape;
  allocateMemory();
 }
}

void Matrix::copyHostToDevice() {
 if (device_allocated && host_allocated) {
  cudaMemcpy(data_device.get(), data_host.get(), shape.x * shape.y * sizeof(float), cudaMemcpyHostToDevice);
  NNException::throwIfDeviceErrorsOccurred("Cannot copy host data to CUDA device.");
 }
 else {
  throw NNException("Cannot copy host data to not allocated memory on device.");
 }
}

void Matrix::copyDeviceToHost() {
 if (device_allocated && host_allocated) {
  cudaMemcpy(data_host.get(), data_device.get(), shape.x * shape.y * sizeof(float), cudaMemcpyDeviceToHost);
  NNException::throwIfDeviceErrorsOccurred("Cannot copy device data to host.");
 }
 else {
  throw NNException("Cannot copy device data to not allocated memory on host.");
 }
}

float& Matrix::operator[](const int index) {
 return data_host.get()[index];
}

const float& Matrix::operator[](const int index) const {
 return data_host.get()[index];
}


Writing matrix.cu


In [14]:
%%writefile main.cu
#include <iostream>
#include "matrix.hh"
#include "nn_exception.hh"

int main() {
    // Create a Matrix object with dimensions 10x10
    Matrix matrix(10, 10);

    // Allocate memory on both host and device
    matrix.allocateMemory();
    std::cout << "Memory allocated on host and device." << std::endl;

    // Initialize host data
    for (size_t i = 0; i < 100; ++i) {
        matrix[i] = static_cast<float>(i);
    }
    std::cout << "Host data initialized." << std::endl;

    // Copy data from host to device
    matrix.copyHostToDevice();
    std::cout << "Data copied from host to device." << std::endl;

    // Clear host data
    for (size_t i = 0; i < 100; ++i) {
        matrix[i] = 0.0f;
    }
    std::cout << "Host data cleared." << std::endl;

    // Copy data back from device to host
    matrix.copyDeviceToHost();
    std::cout << "Data copied from device to host." << std::endl;

    // Verify the data
    bool success = true;
    for (size_t i = 0; i < 100; ++i) {
        if (matrix[i] != static_cast<float>(i)) {
            success = false;
            break;
        }
    }

    if (success) {
        std::cout << "Test passed: Data verification successful." << std::endl;
    } else {
        std::cout << "Test failed: Data verification unsuccessful." << std::endl;
    }

    return 0;
}

Overwriting main.cu


In [15]:
!nvcc main.cu matrix.cu shape.cu -o matrix.out

In [16]:
!./matrix.out

Memory allocated on host and device.
Host data initialized.
Data copied from host to device.
Host data cleared.
Data copied from device to host.
Test passed: Data verification successful.


###Loss Function - Binary Cross Entropy in CUDA

In [17]:
%%writefile bce_cost.hh
#pragma once
#include "matrix.hh"

class BCECost {
public:
 float cost(Matrix predictions, Matrix target);
 Matrix dCost(Matrix predictions, Matrix target, Matrix dY);
};

Writing bce_cost.hh


Two Kernels for calculating BCE in CUDA

In [18]:
%%writefile bce_cost.cu
#include "bce_cost.hh"
#include <cmath>
#include <cassert>
#include "nn_exception.hh"

__global__ void binaryCrossEntropyCost(float* predictions, float* target, int size, float* cost) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    if (index < size) {
        float pred = predictions[index];
        pred = fmaxf(fminf(pred, 1.0f - 1e-7), 1e-7);

        float partial_cost = target[index] * logf(pred)
                + (1.0f - target[index]) * logf(1.0f - pred);
        atomicAdd(cost, - partial_cost / size);
    }
}

__global__ void dBinaryCrossEntropyCost(float* predictions, float* target, float* dY,
                                        int size) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    if (index < size) {
        // Clamp predictions to avoid division by zero
        float pred = predictions[index];
        pred = fmaxf(fminf(pred, 1.0f - 1e-7), 1e-7);

        dY[index] = -1.0 * (target[index] / pred - (1 - target[index]) / (1 - pred));
    }
}

//cost (or loss)
float BCECost::cost(Matrix predictions, Matrix target) {
 assert(predictions.shape.x == target.shape.x);

 float* cost;
 cudaMallocManaged(&cost, sizeof(float));
 *cost = 0.0f;

 dim3 block_size(256);
 dim3 num_of_blocks((predictions.shape.x + block_size.x - 1) / block_size.x);
 binaryCrossEntropyCost<<<num_of_blocks, block_size>>>(predictions.data_device.get(),
                target.data_device.get(),
                predictions.shape.x, cost);
 cudaDeviceSynchronize();
 NNException::throwIfDeviceErrorsOccurred("Cannot compute binary cross entropy cost.");

 float cost_value = *cost;
 cudaFree(cost);

 return cost_value;
}

//derivative of cost (aka derivative of loss)
Matrix BCECost::dCost(Matrix predictions, Matrix target, Matrix dY) {
 assert(predictions.shape.x == target.shape.x);

 dim3 block_size(256);
 dim3 num_of_blocks((predictions.shape.x + block_size.x - 1) / block_size.x);
 dBinaryCrossEntropyCost<<<num_of_blocks, block_size>>>(predictions.data_device.get(),
                 target.data_device.get(),
                 dY.data_device.get(),
                 predictions.shape.x);
 NNException::throwIfDeviceErrorsOccurred("Cannot compute derivative for binary cross entropy.");

 return dY;
}

Writing bce_cost.cu


In [19]:
%%writefile main.cu
#include <iostream>
#include <vector>
#include "matrix.hh"
#include "bce_cost.hh"
#include "nn_exception.hh"

// Helper function to initialize a Matrix with data
void initializeMatrix(Matrix& matrix, const std::vector<float>& data) {
    for (size_t i = 0; i < data.size(); ++i) {
        matrix[i] = data[i];
    }
    matrix.copyHostToDevice();
}

int main() {
    // Define the size of the data
    const int size = 10;

    // Create predictions and target data
    std::vector<float> predictions_data = {0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95};
    std::vector<float> target_data = {0, 0, 1, 0, 1, 0, 1, 1, 1, 0};

    // Create Matrix objects for predictions and targets
    Matrix predictions(size, 1);
    Matrix target(size, 1);
    predictions.allocateMemory();
    target.allocateMemory();

    // Initialize matrices with data
    initializeMatrix(predictions, predictions_data);
    initializeMatrix(target, target_data);

    // Compute the binary cross-entropy cost
    BCECost bce_cost;
    float cost_value = bce_cost.cost(predictions, target);
    std::cout << "Binary Cross-Entropy Cost: " << cost_value << std::endl;

    // Compute the gradient of the binary cross-entropy cost
    Matrix dY(size, 1);
    dY.allocateMemory();
    Matrix dCost_matrix = bce_cost.dCost(predictions, target, dY);
    dCost_matrix.copyDeviceToHost();

    // Print the gradient values
    std::cout << "Gradient of Binary Cross-Entropy Cost: ";
    for (int i = 0; i < size; ++i) {
        std::cout << dCost_matrix[i] << " ";
    }
    std::cout << std::endl;

    return 0;
}

Overwriting main.cu


In [20]:
!nvcc main.cu matrix.cu shape.cu bce_cost.cu -o bce.out

In [21]:
!./bce.out

Binary Cross-Entropy Cost: 0.733365
Gradient of Binary Cross-Entropy Cost: 1.11111 1.25 -3.33333 1.66667 -2 2.5 -1.42857 -1.25 -1.11111 20 


###Defining the Model

Abstract class that will be inherited by all other NN-Layer classes

In [22]:
%%writefile nn_layer.hh
#pragma once

#include <iostream>
#include "matrix.hh"

class NNLayer {
  protected:
    std::string name;

  public:
    virtual ~NNLayer() { }
    virtual Matrix& forward(Matrix& A) = 0;
    virtual Matrix& backprop(Matrix& dZ, float learning_rate) = 0;

    std::string getName() { return this->name; };
};



Writing nn_layer.hh


Linear Layer

In [23]:
%%writefile linear_layer.hh
#pragma once
#include "nn_layer.hh"

class LinearLayer : public NNLayer {
  private:
    const float weights_init_threshold = 0.01;

    Matrix W;
    Matrix b;
    Matrix Z;
    Matrix A;
    Matrix dA;

    void initializeBiasWithZeros();
    void initializeWeightsRandomly();

    void computeAndStoreBackpropError(Matrix& dZ);
    void computeAndStoreLayerOutput(Matrix& A);
    void updateWeights(Matrix& dZ, float learning_rate);
    void updateBias(Matrix& dZ, float learning_rate);

  public:
    LinearLayer(std::string name, Shape W_shape);
    virtual ~LinearLayer() override;

    Matrix& forward(Matrix& A) override;
    Matrix& backprop(Matrix& dZ, float learning_rate=0.01) override;

    int getXDim() const;
    int getYDim() const;

    Matrix& getWeightsMatrix();
    Matrix& getBiasVector();
};


Writing linear_layer.hh


In [24]:
%%writefile linear_layer.cu
#include <iostream>
#include <random>
#include <cassert>
#include <cmath>
#include <algorithm>
#include "linear_layer.hh"
#include "nn_exception.hh"

__global__ void linearLayerForward( float* W, float* A, float* Z, float* b,
									int W_x_dim, int W_y_dim,
									int A_x_dim, int A_y_dim) {

	int row = blockIdx.y * blockDim.y + threadIdx.y;
	int col = blockIdx.x * blockDim.x + threadIdx.x;

	int Z_x_dim = A_x_dim;
	int Z_y_dim = W_y_dim;

	float Z_value = 0;

	if (row < Z_y_dim && col < Z_x_dim) {
		for (int i = 0; i < W_x_dim; i++) {
			Z_value += W[row * W_x_dim + i] * A[i * A_x_dim + col];
		}
		Z[row * Z_x_dim + col] = Z_value + b[row];
	}
}


__global__ void linearLayerBackprop(float* W, float* dZ, float *dA,
									int W_x_dim, int W_y_dim,
									int dZ_x_dim, int dZ_y_dim) {

	int col = blockIdx.x * blockDim.x + threadIdx.x;
	int row = blockIdx.y * blockDim.y + threadIdx.y;

	// W is treated as transposed
	int dA_x_dim = dZ_x_dim;
	int dA_y_dim = W_x_dim;

	float dA_value = 0.0f;

	if (row < dA_y_dim && col < dA_x_dim) {
		for (int i = 0; i < W_y_dim; i++) {
			dA_value += W[i * W_x_dim + row] * dZ[i * dZ_x_dim + col];
		}
		dA[row * dA_x_dim + col] = dA_value;
	}
}

__global__ void linearLayerUpdateWeights(  float* dZ, float* A, float* W,
										   int dZ_x_dim, int dZ_y_dim,
										   int A_x_dim, int A_y_dim,
										   float learning_rate) {

	int col = blockIdx.x * blockDim.x + threadIdx.x;
	int row = blockIdx.y * blockDim.y + threadIdx.y;

	// A is treated as transposed
	int W_x_dim = A_y_dim;
	int W_y_dim = dZ_y_dim;

	float dW_value = 0.0f;

	if (row < W_y_dim && col < W_x_dim) {
		for (int i = 0; i < dZ_x_dim; i++) {
			dW_value += dZ[row * dZ_x_dim + i] * A[col * A_x_dim + i];
		}
		W[row * W_x_dim + col] = W[row * W_x_dim + col] - learning_rate * (dW_value / A_x_dim);
	}
}

__global__ void linearLayerUpdateBias(  float* dZ, float* b,
										int dZ_x_dim, int dZ_y_dim,
										int b_x_dim,
										float learning_rate) {
	int index = blockIdx.x * blockDim.x + threadIdx.x;

	if (index < dZ_x_dim * dZ_y_dim) {
		int dZ_x = index % dZ_x_dim;
		int dZ_y = index / dZ_x_dim;
		atomicAdd(&b[dZ_y], - learning_rate * (dZ[dZ_y * dZ_x_dim + dZ_x] / dZ_x_dim));
	}
}

LinearLayer::LinearLayer(std::string name, Shape W_shape) :
	W(W_shape), b(W_shape.y, 1)
{
	this->name = name;
	b.allocateMemory();
	W.allocateMemory();
	initializeBiasWithZeros();
	initializeWeightsRandomly();
}

LinearLayer::~LinearLayer()
{ }

void LinearLayer::initializeWeightsRandomly() {
	std::default_random_engine generator;
	std::normal_distribution normal_distribution(0.0, 1.0);

	for (int x = 0; x < W.shape.x; x++) {
		for (int y = 0; y < W.shape.y; y++) {
			W[y * W.shape.x + x] = normal_distribution(generator) * weights_init_threshold;
		}
	}

	W.copyHostToDevice();
}

void LinearLayer::initializeBiasWithZeros() {
	for (int x = 0; x < b.shape.x; x++) {
		b[x] = 0;
	}

	b.copyHostToDevice();
}

Matrix& LinearLayer::forward(Matrix& A) {
	assert(W.shape.x == A.shape.y);

	this->A = A;
	Shape Z_shape(A.shape.x, W.shape.y);
	Z.allocateMemoryIfNotAllocated(Z_shape);

	computeAndStoreLayerOutput(A);
	NNException::throwIfDeviceErrorsOccurred("Cannot perform linear layer forward propagation.");

	return Z;
}

void LinearLayer::computeAndStoreLayerOutput(Matrix& A) {
    dim3 block_size(8, 8);
    dim3 num_of_blocks(	(Z.shape.x + block_size.x - 1) / block_size.x,
						(Z.shape.y + block_size.y - 1) / block_size.y);
    linearLayerForward<<<num_of_blocks, block_size>>>(W.data_device.get(),
                                                      A.data_device.get(),
                                                      Z.data_device.get(),
                                                      b.data_device.get(),
                                                      W.shape.x, W.shape.y,
                                                      A.shape.x, A.shape.y);
}

Matrix& LinearLayer::backprop(Matrix& dZ, float learning_rate) {
	dA.allocateMemoryIfNotAllocated(A.shape);

	computeAndStoreBackpropError(dZ);
	NNException::throwIfDeviceErrorsOccurred("Cannot perform back propagation.");

	updateBias(dZ, learning_rate);
	NNException::throwIfDeviceErrorsOccurred("Cannot perform bias update.");

	updateWeights(dZ, learning_rate);
	NNException::throwIfDeviceErrorsOccurred("Cannot perform weights update.");

	return dA;
}

void LinearLayer::computeAndStoreBackpropError(Matrix& dZ) {
    dim3 block_size(8, 8);
    dim3 num_of_blocks(	(A.shape.x + block_size.x - 1) / block_size.x,
						(A.shape.y + block_size.y - 1) / block_size.y);
    linearLayerBackprop<<<num_of_blocks, block_size>>>(W.data_device.get(),
                                                       dZ.data_device.get(),
                                                       dA.data_device.get(),
                                                       W.shape.x, W.shape.y,
                                                       dZ.shape.x, dZ.shape.y);
}

void LinearLayer::updateWeights(Matrix& dZ, float learning_rate) {
    dim3 block_size(8, 8);
    dim3 num_of_blocks(	(W.shape.x + block_size.x - 1) / block_size.x,
						(W.shape.y + block_size.y - 1) / block_size.y);
    linearLayerUpdateWeights<<<num_of_blocks, block_size>>>(dZ.data_device.get(),
                                                            A.data_device.get(),
                                                            W.data_device.get(),
                                                            dZ.shape.x, dZ.shape.y,
                                                            A.shape.x, A.shape.y,
                                                            learning_rate);
}

void LinearLayer::updateBias(Matrix& dZ, float learning_rate) {
    dim3 block_size(256);
    dim3 num_of_blocks( (dZ.shape.y * dZ.shape.x + block_size.x - 1) / block_size.x);
    linearLayerUpdateBias<<<num_of_blocks, block_size>>>(dZ.data_device.get(),
                                                         b.data_device.get(),
                                                         dZ.shape.x, dZ.shape.y,
                                                         b.shape.x, learning_rate);
}

int LinearLayer::getXDim() const {
	return W.shape.x;
}

int LinearLayer::getYDim() const {
	return W.shape.y;
}

Matrix& LinearLayer::getWeightsMatrix() {
    return W;
}

Matrix& LinearLayer::getBiasVector() {
    return b;
}

Writing linear_layer.cu


####main

In [25]:
%%writefile main.cu
#include "linear_layer.hh"
#include "bce_cost.hh"
#include "matrix.hh"

void printMatrix(Matrix& matrix, const std::string& name) {
    matrix.copyDeviceToHost();
    std::cout << name << ":" << std::endl;
    for (int i = 0; i < matrix.shape.x * matrix.shape.y; ++i) {
        std::cout << matrix[i] << " ";
    }
    std::cout << std::endl;
}

int main() {
    // Define input dimensions and initialize the layer
    Shape input_shape(1, 3); // (1 rows, 3 columns, transposed vector)
    Shape weight_shape(3, 1); // shape of weights, resulting in a 1x1 output

    LinearLayer layer("test_layer", weight_shape);

    // Allocate memory for input and output
    Matrix input(input_shape);
    input.allocateMemory();
    input[0] = 0.1f; input[1] = 0.2f; input[2] = 0.3f;
    input.copyHostToDevice();

    // Allocate memory for target
    Matrix target(Shape(1, 1)); // 1x1 target matrix
    target.allocateMemory();
    target[0] = 0.0f;
    target.copyHostToDevice();

    // Print initial weights and biases
    printMatrix(layer.getWeightsMatrix(), "Initial Weights");
    printMatrix(layer.getBiasVector(), "Initial Biases");

    // Perform forward pass
    Matrix& output = layer.forward(input);
    output.copyDeviceToHost();

    // Print forward pass output
    std::cout << "Forward pass output:" << std::endl;
    for (int i = 0; i < output.shape.x * output.shape.y; ++i) {
        std::cout << output[i] << " ";
    }
    std::cout << std::endl;

    // Calculate BCE loss
    BCECost bce;
    float loss = bce.cost(output, target);
    std::cout << "Binary Cross Entropy Loss: " << loss << std::endl;

    // Calculate gradient of BCE loss
    Matrix dZ(output.shape);
    dZ.allocateMemory();
    bce.dCost(output, target, dZ);

    // Perform backpropagation
    float learning_rate = 0.01f;
    Matrix& dA = layer.backprop(dZ, learning_rate);
    dA.copyDeviceToHost();

    // Print backpropagation output (dA)
    std::cout << "Backpropagation output (dA):" << std::endl;
    for (int i = 0; i < dA.shape.x * dA.shape.y; ++i) {
        std::cout << dA[i] << " ";
    }
    std::cout << std::endl;

    // Print updated weights and biases
    printMatrix(layer.getWeightsMatrix(), "Updated Weights");
    printMatrix(layer.getBiasVector(), "Updated Biases");

    return 0;
}


Overwriting main.cu


In [26]:
!nvcc main.cu matrix.cu shape.cu bce_cost.cu linear_layer.cu -o ll.out
!./ll.out

Initial Weights:
-0.00121966 -0.0108682 0.0068429 
Initial Biases:
0 
Forward pass output:
-0.000242732 
Binary Cross Entropy Loss: 1.19209e-07
Backpropagation output (dA):
-0.00121966 -0.0108682 0.0068429 
Updated Weights:
-0.00221966 -0.0128682 0.0038429 
Updated Biases:
-0.01 


###Train a Naive Linear Model + Quick Check

because BCE is clamped, this will attempt to predict a number below zero if the target is zero, and a value above one if the target is 1

In [29]:
%%writefile main.cu
#include "linear_layer.hh"
#include "bce_cost.hh"
#include "matrix.hh"

void printMatrix(Matrix& matrix, const std::string& name) {
    matrix.copyDeviceToHost();
    std::cout << name << ":" << std::endl;
    for (int i = 0; i < matrix.shape.x * matrix.shape.y; ++i) {
        std::cout << matrix[i] << " ";
    }
    std::cout << std::endl;
}

int main() {
    // Define input dimensions and initialize the layer
    Shape input_shape(1, 3); // (1 rows, 3 columns, transposed vector)
    Shape weight_shape(3, 1); // shape of weights, resulting in a 1x1 output

    LinearLayer layer("test_layer", weight_shape);

    // Allocate memory for input and output
    Matrix input(input_shape);
    input.allocateMemory();
    input[0] = 0.1f; input[1] = 0.2f; input[2] = 0.3f;
    input.copyHostToDevice();

    // Allocate memory for target
    Matrix target(Shape(1, 1)); // 1x1 target matrix
    target.allocateMemory();
    target[0] = 0.0f;
    target.copyHostToDevice();

    // Print initial weights and biases
    printMatrix(layer.getWeightsMatrix(), "Initial Weights");
    printMatrix(layer.getBiasVector(), "Initial Biases");

    // Training loop
    for (int i = 0; i < 3; ++i) {
        // Perform forward pass
        Matrix& output = layer.forward(input);
        output.copyDeviceToHost();

        // Print forward pass output
        std::cout << "Forward pass output:" << std::endl;
        for (int j = 0; j < output.shape.x * output.shape.y; ++j) {
            std::cout << output[j] << " ";
        }
        std::cout << std::endl;

        // Calculate BCE loss
        BCECost bce;
        float loss = bce.cost(output, target);
        std::cout << "Loss at iteration " << i << ": " << loss << std::endl;

        // Calculate gradient of BCE loss
        Matrix dZ(output.shape);
        dZ.allocateMemory();
        bce.dCost(output, target, dZ);

        // Perform backpropagation
        float learning_rate = 0.000001f;
        layer.backprop(dZ, learning_rate);
    }

    // Print updated weights and biases
    printMatrix(layer.getWeightsMatrix(), "Updated Weights");
    printMatrix(layer.getBiasVector(), "Updated Biases");

    return 0;
}

Overwriting main.cu


In [30]:
!nvcc main.cu matrix.cu shape.cu bce_cost.cu linear_layer.cu -o ll.out
!./ll.out

Initial Weights:
-0.00121966 -0.0108682 0.0068429 
Initial Biases:
0 
Forward pass output:
-0.000242732 
Loss at iteration 0: 1.19209e-07
Forward pass output:
-0.000243872 
Loss at iteration 1: 1.19209e-07
Forward pass output:
-0.000245012 
Loss at iteration 2: 1.19209e-07
Updated Weights:
-0.00121996 -0.0108688 0.006842 
Updated Biases:
-3e-06 


###Sigmoid Function

In [35]:
%%writefile sigmoid_activation.hh
#pragma once

#include "nn_layer.hh"

class SigmoidActivation : public NNLayer {
private:
	Matrix A;

	Matrix Z;
	Matrix dZ;

public:
	SigmoidActivation(std::string name);
	~SigmoidActivation();

	Matrix& forward(Matrix& Z);
	Matrix& backprop(Matrix& dA, float learning_rate = 0.01);
};

Writing sigmoid_activation.hh


In [42]:
%%writefile sigmoid_activation.cu
#include "sigmoid_activation.hh"
#include "nn_exception.hh"
#include "linear_layer.hh"

#include <iostream>
#include <random>
#include <cassert>
#include <cmath>
#include <algorithm>
#include "sigmoid_activation.hh"

__device__ float sigmoid(float x) {
    return 1.0f / (1 + exp(-x));
}

__global__ void sigmoidActivationForward(float* Z, float* A,
                                         int Z_x_dim, int Z_y_dim) {

    int index = blockIdx.x * blockDim.x + threadIdx.x;

    if (index < Z_x_dim * Z_y_dim) {
        A[index] = sigmoid(Z[index]);
    }
}

__global__ void sigmoidActivationBackprop(float* Z, float* dA, float* dZ,
                                          int Z_x_dim, int Z_y_dim) {

    int index = blockIdx.x * blockDim.x + threadIdx.x;

    if (index < Z_x_dim * Z_y_dim) {
        dZ[index] = dA[index] * sigmoid(Z[index]) * (1 - sigmoid(Z[index]));
    }
}

SigmoidActivation::SigmoidActivation(std::string name) {
    this->name = name;
}

SigmoidActivation::~SigmoidActivation()
{ }

Matrix& SigmoidActivation::forward(Matrix& Z) {
    this->Z = Z;
    A.allocateMemoryIfNotAllocated(Z.shape);

    dim3 block_size(256);
    dim3 num_of_blocks((Z.shape.y * Z.shape.x + block_size.x - 1) / block_size.x);

    sigmoidActivationForward<<<num_of_blocks, block_size>>>(Z.data_device.get(), A.data_device.get(),
                                                            Z.shape.x, Z.shape.y);
    NNException::throwIfDeviceErrorsOccurred("Cannot perform sigmoid forward propagation.");

    return A;
}

Matrix& SigmoidActivation::backprop(Matrix& dA, float learning_rate) {
    dZ.allocateMemoryIfNotAllocated(Z.shape);

    dim3 block_size(256);
    dim3 num_of_blocks((Z.shape.y * Z.shape.x + block_size.x - 1) / block_size.x);
    sigmoidActivationBackprop<<<num_of_blocks, block_size>>>(Z.data_device.get(), dA.data_device.get(),
                                                             dZ.data_device.get(),
                                                             Z.shape.x, Z.shape.y);
    NNException::throwIfDeviceErrorsOccurred("Cannot perform sigmoid back propagation");

    return dZ;
}


Overwriting sigmoid_activation.cu


In [43]:
%%writefile main.cu
#include "sigmoid_activation.hh"
#include "nn_exception.hh"
#include "matrix.hh"
#include "linear_layer.hh"

void printMatrix(Matrix& matrix, const std::string& name) {
    matrix.copyDeviceToHost();
    std::cout << name << ":" << std::endl;
    for (int i = 0; i < matrix.shape.x * matrix.shape.y; ++i) {
        std::cout << matrix[i] << " ";
    }
    std::cout << std::endl;
}

int main() {
    // Define input dimensions and initialize the matrix
    Shape input_shape(1, 3); // (1 rows, 3 columns)

    // Initialize SigmoidActivation
    SigmoidActivation sigmoid("sigmoid_activation");

    // Allocate memory for input matrix
    Matrix input(input_shape);
    input.allocateMemory();
    input[0] = -1.0f; input[1] = 0.0f; input[2] = 1.0f;
    input.copyHostToDevice();

    // Perform forward pass
    Matrix& output = sigmoid.forward(input);
    output.copyDeviceToHost();

    // Print forward pass output
    printMatrix(output, "Forward pass output");

    // Allocate memory for gradient matrix
    Matrix dA(output.shape);
    dA.allocateMemory();
    dA[0] = 0.1f; dA[1] = 0.2f; dA[2] = 0.3f;
    dA.copyHostToDevice();

    // Perform backward pass
    Matrix& dZ = sigmoid.backprop(dA, 0.01f);
    dZ.copyDeviceToHost();

    // Print backward pass output
    printMatrix(dZ, "Backward pass output");

    return 0;
}

Overwriting main.cu


In [44]:
!nvcc main.cu matrix.cu shape.cu bce_cost.cu sigmoid_activation.cu -o sig.out
!./sig.out

Forward pass output:
0.268941 0.5 0.731059 
Backward pass output:
0.0196612 0.05 0.0589836 


###ReLU Activation

In [45]:
%%writefile relu_activation.hh
#pragma once

#include "nn_layer.hh"

class ReLUActivation : public NNLayer {
private:
	Matrix A;

	Matrix Z;
	Matrix dZ;

public:
	ReLUActivation(std::string name);
	~ReLUActivation();

	Matrix& forward(Matrix& Z);
	Matrix& backprop(Matrix& dA, float learning_rate = 0.01);
};

Writing relu_activation.hh


In [51]:
%%writefile relu_activation.cu
#include "relu_activation.hh"
#include "nn_exception.hh"

__global__ void reluActivationForward(float* Z, float* A,
                                      int Z_x_dim, int Z_y_dim) {

    int index = blockIdx.x * blockDim.x + threadIdx.x;

    if (index < Z_x_dim * Z_y_dim) {
        A[index] = fmaxf(Z[index], 0);
    }
}

__global__ void reluActivationBackprop(float* Z, float* dA, float* dZ,
                                       int Z_x_dim, int Z_y_dim) {

    int index = blockIdx.x * blockDim.x + threadIdx.x;

    if (index < Z_x_dim * Z_y_dim) {
        if (Z[index] > 0) {
            dZ[index] = dA[index];
        }
        else {
            dZ[index] = 0;
        }
    }
}

ReLUActivation::ReLUActivation(std::string name) {
    this->name = name;
}

ReLUActivation::~ReLUActivation() { }

Matrix& ReLUActivation::forward(Matrix& Z) {
    this->Z = Z;
    A.allocateMemoryIfNotAllocated(Z.shape);

    dim3 block_size(256);
    dim3 num_of_blocks((Z.shape.y * Z.shape.x + block_size.x - 1) / block_size.x);

    reluActivationForward<<<num_of_blocks, block_size>>>(Z.data_device.get(), A.data_device.get(),
                                                         Z.shape.x, Z.shape.y);
    NNException::throwIfDeviceErrorsOccurred("Cannot perform ReLU forward propagation.");

    return A;
}

Matrix& ReLUActivation::backprop(Matrix& dA, float learning_rate) {
    dZ.allocateMemoryIfNotAllocated(Z.shape);

    dim3 block_size(256);
    dim3 num_of_blocks((Z.shape.y * Z.shape.x + block_size.x - 1) / block_size.x);
    reluActivationBackprop<<<num_of_blocks, block_size>>>(Z.data_device.get(), dA.data_device.get(),
                                                          dZ.data_device.get(),
                                                          Z.shape.x, Z.shape.y);
    NNException::throwIfDeviceErrorsOccurred("Cannot perform ReLU back propagation");

    return dZ;
}


Overwriting relu_activation.cu


In [52]:
%%writefile main.cu
#include "relu_activation.hh"
#include "nn_exception.hh"
#include "matrix.hh"
#include "sigmoid_activation.hh"
#include "linear_layer.hh"

void printMatrix(Matrix& matrix, const std::string& name) {
    matrix.copyDeviceToHost();
    std::cout << name << ":" << std::endl;
    for (int i = 0; i < matrix.shape.x * matrix.shape.y; ++i) {
        std::cout << matrix[i] << " ";
    }
    std::cout << std::endl;
}

int main() {
    // Define input dimensions and initialize the matrix
    Shape input_shape(1, 3); // (1 rows, 3 columns)

    // Initialize ReLUActivation
    ReLUActivation relu("relu_activation");

    // Allocate memory for input matrix
    Matrix input(input_shape);
    input.allocateMemory();
    input[0] = -1.0f; input[1] = 0.0f; input[2] = 1.0f;
    input.copyHostToDevice();

    // Perform forward pass
    Matrix& output = relu.forward(input);
    output.copyDeviceToHost();

    // Print forward pass output
    printMatrix(output, "Forward pass output");

    // Allocate memory for gradient matrix
    Matrix dA(output.shape);
    dA.allocateMemory();
    dA[0] = 0.1f; dA[1] = 0.2f; dA[2] = 0.3f;
    dA.copyHostToDevice();

    // Perform backward pass
    Matrix& dZ = relu.backprop(dA, 0.01f);
    dZ.copyDeviceToHost();

    // Print backward pass output
    printMatrix(dZ, "Backward pass output");

    return 0;
}

Overwriting main.cu


In [53]:
!nvcc main.cu matrix.cu shape.cu bce_cost.cu relu_activation.cu -o sig.out
!./sig.out

Forward pass output:
0 0 1 
Backward pass output:
0 0 0.3 


##Putting everything together

Now we can put this all together to train a model. To do that we'll define two more (very small) abstractions–one for the model and one for the dataset

###Neural Network

In [77]:
%%writefile neural_network.hh
#pragma once

#include "nn_layer.hh"
#include <vector>
#include "bce_cost.hh"
#include "matrix.hh" // Ensure Matrix is defined or included if it's a separate file.

class NeuralNetwork {
private:
    std::vector<NNLayer*> layers; // Specify the type of elements in the vector
    BCECost bce_cost;

    Matrix Y;
    Matrix dY;
    float learning_rate;

public:
    NeuralNetwork(float learning_rate = 0.01);
    ~NeuralNetwork();

    Matrix forward(Matrix X);
    void backprop(Matrix predictions, Matrix target);

    void addLayer(NNLayer *layer);
    std::vector<NNLayer*> getLayers() const; // Return a vector of pointers to NNLayer
};


Overwriting neural_network.hh


In [84]:
%%writefile neural_network.cu
#include "neural_network.hh"
#include "nn_exception.hh"
#include <vector>

NeuralNetwork::NeuralNetwork(float learning_rate) :
    learning_rate(learning_rate)
{ }

NeuralNetwork::~NeuralNetwork() {
    for (auto layer : layers) {
        delete layer;
    }
}

void NeuralNetwork::addLayer(NNLayer* layer) {
    this->layers.push_back(layer);
}

Matrix NeuralNetwork::forward(Matrix X) {
    Matrix Z = X;

    for (auto layer : layers) {
        Z = layer->forward(Z);
    }

    Y = Z;
    return Y;
}

void NeuralNetwork::backprop(Matrix predictions, Matrix target) {
    dY.allocateMemoryIfNotAllocated(predictions.shape);
    Matrix error = bce_cost.dCost(predictions, target, dY);

    for (auto it = this->layers.rbegin(); it != this->layers.rend(); ++it) {
        error = (*it)->backprop(error, learning_rate);
    }

    cudaDeviceSynchronize();
}

std::vector<NNLayer*> NeuralNetwork::getLayers() const {
    return layers;
}


Overwriting neural_network.cu


###Dataset

In [78]:
%%writefile coordinates_dataset.hh
#pragma once

#include "matrix.hh"
#include "shape.hh"
#include <vector>
#include <cstdlib>
#include <ctime>

class CoordinatesDataset {
private:
    size_t batch_size;
    size_t number_of_batches;

    std::vector<Matrix> batches; // Specify the type of elements in the vector
    std::vector<Matrix> targets; // Specify the type of elements in the vector

public:
    CoordinatesDataset(size_t batch_size, size_t number_of_batches);

    int getNumOfBatches();
    std::vector<Matrix>& getBatches(); // Return reference to vector of Matrix
    std::vector<Matrix>& getTargets(); // Return reference to vector of Matrix
};


Overwriting coordinates_dataset.hh


In [81]:
%%writefile coordinates_dataset.cu
#include "coordinates_dataset.hh"
#include <vector>

CoordinatesDataset::CoordinatesDataset(size_t batch_size, size_t number_of_batches) :
    batch_size(batch_size), number_of_batches(number_of_batches)
{
    for (int i = 0; i < number_of_batches; i++) {
        batches.push_back(Matrix(Shape(batch_size, 2)));
        targets.push_back(Matrix(Shape(batch_size, 1)));

        batches[i].allocateMemory();
        targets[i].allocateMemory();

        for (int k = 0; k < batch_size; k++) {
            batches[i][k] = static_cast<float>(rand()) / RAND_MAX - 0.5;
            batches[i][batches[i].shape.x + k] = static_cast<float>(rand()) / RAND_MAX - 0.5;

            if ((batches[i][k] > 0 && batches[i][batches[i].shape.x + k] > 0) ||
                (batches[i][k] < 0 && batches[i][batches[i].shape.x + k] < 0)) {
                targets[i][k] = 1;
            }
            else {
                targets[i][k] = 0;
            }
        }

        batches[i].copyHostToDevice();
        targets[i].copyHostToDevice();
    }
}

int CoordinatesDataset::getNumOfBatches() {
    return number_of_batches;
}

std::vector<Matrix>& CoordinatesDataset::getBatches() {
    return batches;
}

std::vector<Matrix>& CoordinatesDataset::getTargets() {
    return targets;
}


Overwriting coordinates_dataset.cu


###Training NN on the dataset, using the model, and evaluating it

In [85]:
%%writefile main.cu
#include "matrix.hh"
#include "shape.hh"
#include "neural_network.hh"
#include "linear_layer.hh"
#include "relu_activation.hh"
#include "sigmoid_activation.hh"
#include "nn_exception.hh"
#include "bce_cost.hh"
#include "coordinates_dataset.hh"
#include <vector>
#include <ctime> // Include for time()
#include <iostream> // Include for std::cout

float computeAccuracy(const Matrix& predictions, const Matrix& targets);

int main() {
    srand(static_cast<unsigned int>(time(NULL))); // Corrected for proper use of time()

    CoordinatesDataset dataset(100, 21);
    BCECost bce_cost;

    NeuralNetwork nn;
    nn.addLayer(new LinearLayer("linear_1", Shape(2, 30)));
    nn.addLayer(new ReLUActivation("relu_1"));
    nn.addLayer(new LinearLayer("linear_2", Shape(30, 1)));
    nn.addLayer(new SigmoidActivation("sigmoid_output"));

    // Network training
    Matrix Y;
    for (int epoch = 0; epoch < 1001; epoch++) {
        float cost = 0.0f;

        for (int batch = 0; batch < dataset.getNumOfBatches() - 1; batch++) {
            Y = nn.forward(dataset.getBatches().at(batch));
            nn.backprop(Y, dataset.getTargets().at(batch));
            cost += bce_cost.cost(Y, dataset.getTargets().at(batch));
        }

        if (epoch % 100 == 0) {
            std::cout << "Epoch: " << epoch
                      << ", Cost: " << cost / dataset.getNumOfBatches()
                      << std::endl;
        }
    }

    // Compute accuracy
    Y = nn.forward(dataset.getBatches().at(dataset.getNumOfBatches() - 1));
    Y.copyDeviceToHost();

    float accuracy = computeAccuracy(
        Y, dataset.getTargets().at(dataset.getNumOfBatches() - 1));
    std::cout << "Accuracy: " << accuracy << std::endl;

    return 0;
}

float computeAccuracy(const Matrix& predictions, const Matrix& targets) {
    int m = predictions.shape.x;
    int correct_predictions = 0;

    for (int i = 0; i < m; i++) {
        float prediction = predictions[i] > 0.5f ? 1.0f : 0.0f;
        if (prediction == targets[i]) {
            correct_predictions++;
        }
    }

    return static_cast<float>(correct_predictions) / m;
}


Overwriting main.cu


In [91]:
!nvcc main.cu matrix.cu shape.cu bce_cost.cu sigmoid_activation.cu relu_activation.cu linear_layer.cu coordinates_dataset.cu neural_network.cu -o main.out
!./main.out

Epoch: 0, Cost: 0.660185
Epoch: 100, Cost: 0.660049
Epoch: 200, Cost: 0.659814
Epoch: 300, Cost: 0.659071
Epoch: 400, Cost: 0.65675
Epoch: 500, Cost: 0.649827
Epoch: 600, Cost: 0.631669
Epoch: 700, Cost: 0.595266
Epoch: 800, Cost: 0.540859
Epoch: 900, Cost: 0.464867
Epoch: 1000, Cost: 0.378224
Accuracy: 0.95


In [95]:
!nvprof ./main.out

==20394== NVPROF is profiling process 20394, command: ./main.out
Epoch: 0, Cost: 0.660178
Epoch: 100, Cost: 0.659858
Epoch: 200, Cost: 0.659594
Epoch: 300, Cost: 0.658729
Epoch: 400, Cost: 0.656002
Epoch: 500, Cost: 0.647854
Epoch: 600, Cost: 0.627464
Epoch: 700, Cost: 0.591304
Epoch: 800, Cost: 0.544744
Epoch: 900, Cost: 0.486751
Epoch: 1000, Cost: 0.412162
Accuracy: 0.88
==20394== Profiling application: ./main.out
==20394== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   83.90%  3.36726s     20020  168.20us  98.304us  8.2207ms  binaryCrossEntropyCost(float*, float*, int, float*)
                    3.86%  154.86ms     40040  3.8670us  3.0390us  7.4890us  linearLayerUpdateWeights(float*, float*, float*, int, int, int, int, float)
                    3.31%  132.88ms     40040  3.3180us  1.3430us  9.1840us  linearLayerUpdateBias(float*, float*, int, int, int, float)
                    2.57%  103.04ms     40042  2.57